In [7]:
import openpyxl
import csv
import json
import requests
import re
import pandas as pd
import time
from time import sleep
import pyautogui as pag
from bs4 import BeautifulSoup
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver import Edge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchAttributeException

In [56]:
options = uc.ChromeOptions()
options.headless = False
options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(version_main=134)
driver.get("https://www.foody.vn")

In [72]:
page_source = BeautifulSoup(driver.page_source, 'html.parser')
shop_address = page_source.find_all('div', 'content-item ng-scope')
# shop_address = page_source.find_all('div', 'title fd-text-ellip')

In [73]:
shop_info = []
shop_lat_lon = []
for shop_href in shop_address:
    # Kiểm tra nếu thẻ <a> tồn tại
    a_tag = shop_href.find('a')
    if a_tag and a_tag.get('href'):  # Đảm bảo 'href' không phải None
        href = a_tag.get('href')
        shop_info.append(f'https://www.foody.vn{href}')
        shop_lat_lon.append(f'https://www.foody.vn{href}/nearby')
print(len(shop_info))
print(len(shop_lat_lon))

341
341


In [ ]:
def getValue(main_restaurant):
    return {
        'Name.1': main_restaurant.get('Name'),
        'Address': main_restaurant.get('Address'),
        'District': main_restaurant.get('District'),
        'City': main_restaurant.get('City'),
        'Latitude': main_restaurant.get('Latitude'),
        'Longitude': main_restaurant.get('Longitude')
    }

In [75]:
def get_working_time(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() 
        page_source = BeautifulSoup(response.text, 'html.parser')
        working_time_div = page_source.find("div", class_="micro-timesopen")
        if working_time_div:
            working_time_spans = working_time_div.find_all("span")
            return working_time_spans[2].text if len(working_time_spans) > 2 else "Null"
        return "Null"
    except (requests.RequestException, Exception) as e:
        return "Null"
    
time_list = [get_working_time(url) for url in shop_info]
time_df = pd.DataFrame(time_list, columns=['Working hours'])
display(time_df)

,Working hours
0,09:00 - 22:00
1,03:30 - 10:30
2,Null
3,Null
4,14:00 - 21:00
...,...
336,06:00 - 22:00
337,07:30 - 22:00
338,07:00 - 22:00
339,07:00 - 22:00


In [77]:
data_list = []
# for index, i in enumerate(shop_lat_lon[700:], start=700):
for index, i in enumerate(shop_lat_lon):
    try:
        response = requests.get(i, timeout=10)
        time.sleep(1)
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', string=lambda x: x and 'var intData' in x)
        if script_tag:
            script_content = script_tag.string
            start = script_content.find('var intData = ') + len('var intData = ')
            end = script_content.find(';', start)
            json_str = script_content[start:end].strip()
            if json_str:
                try:
                    data = json.loads(json_str)
                    main_restaurant = data.get('MainRestaurant', {})
                    new_df = getValue(main_restaurant)
                    data_list.append(new_df)
                    print(index+1)
                except json.JSONDecodeError as e:
                    print(f"JSONDecodeError tại URL {i}: {e}")
                    data_list.append({})
            else:
                print(f"Không tìm thấy chuỗi JSON hợp lệ tại URL {i}")
                data_list.append({})
        else:
            print(f"Không tìm thấy script chứa 'var intData' tại URL {i}")
            data_list.append({})
    except Exception as e:
        print(f"Lỗi khi xử lý URL {i}: {e}")
        data_list.append({})
df = pd.DataFrame(data_list)
display(df)


1
2
Lỗi khi xử lý URL https://www.foody.vnhttps://shopeefood.vn/long-an/lotteria-tan-an/nearby: HTTPSConnectionPool(host='www.foody.vnhttps', port=443): Max retries exceeded with url: /shopeefood.vn/long-an/lotteria-tan-an/nearby (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001AE3DDA9520>, 'Connection to www.foody.vnhttps timed out. (connect timeout=10)'))
Lỗi khi xử lý URL https://www.foody.vnhttps://shopeefood.vn/long-an/nana-tra-sua-thach-nha-lam/nearby: HTTPSConnectionPool(host='www.foody.vnhttps', port=443): Max retries exceeded with url: /shopeefood.vn/long-an/nana-tra-sua-thach-nha-lam/nearby (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001AE3F2408C0>, 'Connection to www.foody.vnhttps timed out. (connect timeout=10)'))
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
Lỗi khi x

,Name,Address,District,City,Latitude,Longitude
0,Trà Sữa Gong Cha - 貢茶 - Vincom Plaza Tân An,"Shop House PG - 09, Vincom Plaza Tân An, Hùng ...",Thành Phố Tân An,Long An,10.534268,106.410453
1,Hủ Tiếu Hoàng Phúc,"140 Quốc Lộ 50, Ấp Chợ Trạm, Xã Mỹ Lệ",Huyện Cần Đước,Long An,10.552677,106.627636
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,Khoai Lang Lắc Xí Muội,"450 Hùng Vương, P. 3",Thành Phố Tân An,Long An,10.531088,106.416849
...,...,...,...,...,...,...
336,Bến My Lăng Cafe,"105 Sương Nguyệt Ánh, P. 2",Thành Phố Tân An,Long An,10.541394,106.403309
337,Ấn Tượng Cafe,"40 Trần Phong Sắc, P. 4",Thành Phố Tân An,Long An,10.533014,106.403929
338,Nob To Go - Cafe & Drinks,3 Trương Văn Bang,Thành Phố Tân An,Long An,10.536832,106.406580
339,New Sky Coffee - Hậu Nghĩa,"Ô6 Khu B, Hậu Nghĩa",Huyện Đức Hòa,Long An,10.907031,106.390534


In [78]:
result = pd.concat([df, time_df], axis=1)
# result.to_csv('ThuDau_2.csv', mode='a',index=False, header=False, encoding='utf-8-sig')
result.to_csv('DATA/Foody_LongAn.csv', mode='a',index=False, encoding='utf-8-sig')
# 